# Basics



There are lots of non-FastHTML-specific tricks and patterns involved in building web apps. The goal of this tutorial is to give an alternate introduction to FastHTML, building out example applications to show common patterns and illustrate some of the ways you can build on top of the FastHTML foundations to create your own custom web apps. A secondary goal is to have this be a useful document to add to the context of an LLM to turn it into a useful FastHTML assistant - in fact, in some of the examples we'll see this kind of assistant in action, thanks to [this custom GPT](https://chatgpt.com/g/g-xPqF9SZjM-fasthtml-helper).

Let's get started.

## FastHTML Basics

FastHTML is *just python*. You can install it with `pip install python-fasthtml`, and extensions/components built for it can likewise be distriuted via pypi or as simple python files.

The core usage of FastHTML is to define routes, and then to define what to do at each route. This is similar to the [FastAPI](https://fastapi.tiangolo.com/) web framework (in fact we implemented much of the fuctionality to match the FastAPI usage examples) but where FastAPI focuses on returning JSON data to build APIs, FastHTML focuses on returning HTML data.

Here's a simple FastHTML app that returns a "Hello, World" message:

In [ ]:
from fasthtml import FastHTML

app = FastHTML()

@app.get("/")
def home():
    return "<h1>Hello, World</h1>"

To run this app, place it in a file, say `app.py`, and then run it with `uvicorn app:app --reload`. You'll see a message like this:
```
INFO:     Will watch for changes in these directories: ['/home/jonathan/fasthtml-example']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [871942] using WatchFiles
INFO:     Started server process [871945]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

If you navigate to http://127.0.0.1:8000 in a browser, you'll see your "Hello, World". If you edit the `app.py` file and save it, the server will reload and you'll see the updated message when you refresh the page in your browser.

## Constructing HTML

Notice we wrote some HTML in the previous example. We don't want to do that! Some web frameworks require that you learn HTML, CSS, Javascript AND some templating language AND python. We want to do as much as possible with just one language. Fortunately, [fastcore.xml](https://fastcore.fast.ai/xml.html) has all we need for constructing HTML from python, and FastHTML includes all the tags you need to get started. For example:

In [ ]:
from fasthtml.common import *
page = Html(
    Head(Title('Some page')),
    Body(Div('Some text, ', A('A link', href='https://example.com'), Img(src="https://placehold.co/200"), cls='myclass')))
print(to_xml(page))

<!doctype html></!doctype>

<html>
  <head>
    <title>Some page</title>
  </head>
  <body>
    <div class="myclass">
Some text, 
      <a href="https://example.com">A link</a>
      <img src="https://placehold.co/200">
    </div>
  </body>
</html>



In [ ]:
show(page)

If that `import *` worries you, you can always import only the tags you need. 

FastHTML is smart enough to know about fastcore.xml, and so you don't need to use the `to_xml` function to convert your XT objects to HTML. You can just return them as you would any other python object. For example, if we modify our previous example to use fastcore.xml, we can return an XT object directly:

In [ ]:
app = FastHTML()

@app.get("/")
def home():
    return Div(H1('Hello, World'), P('Some text'), P('Some more text'))

This will render the HTML in the browser.

For debugging, you can right-click on the rendered HTML in the browser and select "Inspect" to see the underlying HTML that was generated. There you'll also find the 'network' tab, which shows you the requests that were made to render the page. Refresh and look for the request to `127.0.0.1` - and you'll see it's just a `GET` request to `/`, and the response body is the HTML you just returned.

You can also use Starlette's `TestClient` to try it out in a notebook: 

In [ ]:
from starlette.testclient import TestClient
client = TestClient(app)
r = client.get("/")
r.text

'<!doctype html></!doctype>\n\n<html>\n  <head>\n    <title>FastHTML page</title>\n    <meta charset="utf-8"></meta>\n    <meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"></meta>\n    <script src="https://unpkg.com/htmx.org@next/dist/htmx.min.js"></script>\n    <script src="https://cdn.jsdelivr.net/gh/answerdotai/surreal@1.3.0/surreal.js"></script>\n    <script src="https://cdn.jsdelivr.net/gh/gnat/css-scope-inline@main/script.js"></script>\n  </head>\n  <body>\n<div>\n  <h1>Hello, World</h1>\n  <p>Some text</p>\n  <p>Some more text</p>\n</div>\n  </body>\n</html>\n'

FastHTML wraps things in an Html tag if you don't do it yourself (unless the request comes from htmx, in which case you get the element directly). 
See the section 'XT objects and HTML' for more on creating custom components or adding HTML rendering to existing python objects.
To give the page a non-default title, return a Title before your main content:

In [ ]:
app = FastHTML()

@app.get("/")
def home():
    return Title("Page Demo"), Div(H1('Hello, World'), P('Some text'), P('Some more text'))

client = TestClient(app)
print(client.get("/").text)

<!doctype html></!doctype>

<html>
  <head>
    <title>Page Demo</title>
    <meta charset="utf-8"></meta>
    <meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"></meta>
    <script src="https://unpkg.com/htmx.org@next/dist/htmx.min.js"></script>
    <script src="https://cdn.jsdelivr.net/gh/answerdotai/surreal@1.3.0/surreal.js"></script>
    <script src="https://cdn.jsdelivr.net/gh/gnat/css-scope-inline@main/script.js"></script>
  </head>
  <body>
<div>
  <h1>Hello, World</h1>
  <p>Some text</p>
  <p>Some more text</p>
</div>
  </body>
</html>



We'll use this pattern often in the examples to follow.